从test数据出发，构造出测试判断能力的数据集。错误类型分为
- Length not match
- Wrong number 1
- Wrong number 2

In [1]:
input_path = './test_3digit_10000.txt'
Length_not_match_path = './Length_not_match.txt'
Wrong_number_1_path = './Wrong_number_1.txt'
Wrong_number_2_path = './Wrong_number_2.txt'

In [7]:
import re
import math
import random

def randint_exclude(start, end, exclude):
    while True:
        num = random.randint(start, end)
        if num != exclude:
            return num
        
def replace_char_at_index(orig_string, index, new_char):
    new_string = orig_string[:index] + new_char + orig_string[index+1:]
    return new_string

def modify_result(expression, addend, mode, verify:bool = False):
        # 使用正则表达式提取表达式中的数字
        match = re.match(r'(\d+)\+(\d+)=(\d+)', expression)
        if match:
            # 提取数字并计算新的结果
            num1 = int(match.group(1))
            num2 = int(match.group(2))
            result = int(match.group(3))
            num_digit = len(match.group(3).strip())
            if verify:
                flag = 1
            else:
                flag = random.uniform(0,1)
            if mode == 'extra_num':
                if flag>0.5:
                    flag2 = random.uniform(0,1)
                    extra = random.randint(1, 9)
                    if flag2>0.5:
                        new_expression = f"j({num1}+{num2}={extra}{result})~F\n"
                    else:
                        new_expression = f"j({num1}+{num2}={result}{extra})~F\n"
                else:
                    new_expression = f"j({num1}+{num2}={result})~T\n"
            elif mode == 'w2':
                if flag>0.5:
                    char_c = str(result)
                    n = len(char_c)
                    # 决定添加错误的位置
                    wrong_loc = random.randint(0, n-1)
                    changed_number = char_c[wrong_loc]
                    if wrong_loc == 0:
                        addend = randint_exclude(1, 9, int(changed_number))
                    else:
                        addend = randint_exclude(0, 9, int(changed_number))
                                
                    char_c = replace_char_at_index(char_c, wrong_loc, str(addend))
                    new_result = int(char_c)
                    
                    # # 老版：
                    # # 决定添加错误的位置
                    # wrong_loc = num_digit-1
                    # new_result = result + addend * (10**wrong_loc)
                    # 构建新的表达式
                    new_expression = f"j({num1}+{num2}={new_result})~F\n"
                else:
                    new_expression = f"j({num1}+{num2}={result})~T\n"
            elif mode == 'w1':
                if flag>0.5:
                    # 决定添加错误的位置
                    wrong_loc = num_digit-1
                    new_result = result + addend * (10**wrong_loc)
                    # 构建新的表达式
                    new_expression = f"j({num1}+{num2}={new_result})~F\n"
                else:
                    new_expression = f"j({num1}+{num2}={result})~T\n"
            else:
                raise ValueError('Don\'t support this noise mode')
                        
            return new_expression
        
        else:
            raise ValueError("Invalid expression format")
            

def trans2judge_specific_wrong(input_path, output_path, jugde_mode='w1', only_judge=True):
    
    with open(input_path, 'r') as f:
        lines = f.readlines()
        
    with open(output_path, 'w') as f2:
        for line in lines:
            if not only_judge:
                f2.write(line)
            line = line.strip()
            new_line = f'j({line})~T\n'
            f2.write(new_line)
            if random.uniform(0,1) > 0.5:
                if jugde_mode == 'w1':
                    new_line = modify_result(line, 1, 'w1')
                elif jugde_mode == 'w2':
                    new_line = modify_result(line, 1, 'w2')
                elif jugde_mode == 'extra_num':
                    new_line = modify_result(line, 1, 'extra_num')
                else:
                    raise ValueError('Don\'t support this noise mode')
                f2.write(new_line)
                

In [8]:
trans2judge_specific_wrong(input_path, Length_not_match_path, 'extra_num')
trans2judge_specific_wrong(input_path, Wrong_number_1_path, 'w1')
trans2judge_specific_wrong(input_path, Wrong_number_2_path, 'w2')